# MNIST

We see FC and convolutional networks (first time) here.

- FC = Fully Connected

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F # for convolution functions
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary

Here we define a fully connected network with 1 hidden layer and a **sigmoid** activation function

In [3]:
class FCNet(nn.Module):
    def __init__(self,din=784, dh=30, dout=10):
        super(FCNet, self).__init__()
        
        self.lin1 = nn.Linear(din, dh)
        self.lin2 = nn.Linear(dh, dout)
    
    def forward(self, x):    
        x = torch.sigmoid(self.lin1(x)) 
        x = self.lin2(x)
        return F.log_softmax(x, dim=1)

Here we instead define a **convolutional neural network** with
- 2 convolutional layers (filters of size 5x5):
    - the 1st one has 20 nodes (i.e. filters), activation = ReLU, pooling: maxpooling 2d 2x2
    - the 2nd one has 50 nodes (i.e. filters), activation = ReLU, pooling: maxpooling 2d 2x2
- 2 fully connected layers:
    - the 1st one has 500 nodes
    - the 2nd one has 10 nodes and is the output layer of the cNN
    
The convolutional part is the **feature extractor**, the fully connected part is the **classifier**

Also **NOTE** that we are using the **softmax** method to get a **discriminative** model: the outputs will be probabilities of belonging to a certain class.

In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        #data = data.view(-1,784)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [6]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            #data = data.view(-1,784)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
# Training settings
#input_size=(1,784,)
input_size=(1,28,28,)

batch_size=64
test_batch_size=1000
epochs=1
lr=0.01
momentum=0.0   
seed=1
log_interval=100

In [8]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [9]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)

In [10]:
model = ConvNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [11]:
summary(model,input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 24, 24]             520
            Conv2d-2             [-1, 50, 8, 8]          25,050
            Linear-3                  [-1, 500]         400,500
            Linear-4                   [-1, 10]           5,010
Total params: 431,080
Trainable params: 431,080
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.12
Params size (MB): 1.64
Estimated Total Size (MB): 1.76
----------------------------------------------------------------


In [12]:
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313472
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.336843
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.307396
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.302228
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.330740
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.443828
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.368600
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.170816
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.186492
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.108023

Test set: Average loss: 0.1564, Accuracy: 9549/10000 (95%)



# Exercise

- The code will run but the network will not train. 
The problem is that FCNet output is inconsistent with the criterion used.
Modify the code in order for the FCNet to work with cross entropy.

- Then modify the code accordingly to use ConvNet (Duplicate this notebook if you find useful)